In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
from utils.data import Subject, load_participant_list

In [3]:
base_dir = '/Users/hugofluhr/phd_local/data/LearningHabits/dev_sample'
sub_ids = load_participant_list(base_dir)

In [4]:
subjects = [Subject(base_dir, sub_id, include_modeling=True, include_imaging=False) for sub_id in sub_ids]

/Users/hugofluhr/phd_local/repositories/learning-habits-analysis/notebooks/../utils/data.py:178: UserWarning: Last 1 trial(s) of block had no response, filling with 0


In [35]:
#trials = [sub.extended_trials.assign(sub_id=sub.sub_id) for sub in subjects]
#trials = pd.concat(trials).set_index('sub_id', append=True).reorder_levels(['sub_id', None])
trials = subjects[0].extended_trials.copy()

In [36]:
trials.columns

Index(['block', 'left_stim', 'right_stim', 'left_value', 'right_value',
       'shift', 'action', 'rt', 'chosen_stim', 'reward', 'correct',
       't_first_stim', 't_second_stim', 't_action', 't_purple_frame',
       't_points_feedback', 't_iti_onset', 't_trial_end', 'first_stim',
       'second_stim', 'left_stim_name', 'right_stim_name', 'left_stim_cat',
       'right_stim_cat', 'first_stim_name', 'second_stim_name',
       'first_stim_cat', 'second_stim_cat', 'phase', 'run', 'trial2',
       'stim_chosen', 'stim_unchosen', 'stim_rawID_chosen', 'reward1',
       'reward2', 'reward_chosen', 'reward_unchosen', 'diff_val',
       'flag_therapy1', 'flag_accuracy', 'flag_include', 'alpha_rl20',
       'beta_rl20', 'alpha_ck20', 'beta_ck20', 'choice_prob_left',
       'choice_prob_right', 'action_prob', 'stim1_value_rl', 'stim2_value_rl',
       'stim3_value_rl', 'stim4_value_rl', 'stim5_value_rl', 'stim6_value_rl',
       'stim7_value_rl', 'stim8_value_rl', 'stim1_value_ck', 'stim2_value_c

In [37]:
trials['first_stim_choice_val'] = (
    trials['beta_rl20'] * trials['first_stim_value_rl'] +
    trials['beta_ck20'] * trials['first_stim_value_ck']
)
trials['second_stim_choice_val'] = (
    trials['beta_rl20'] * trials['second_stim_value_rl'] +
    trials['beta_ck20'] * trials['second_stim_value_ck']
)

In [38]:
trials['left_stim_choice_val'] = (
    trials['beta_rl20'] * trials.apply(lambda row: row[f'stim{row["left_stim"]}_value_rl'], axis=1) +
    trials['beta_ck20'] * trials.apply(lambda row: row[f'stim{row["left_stim"]}_value_ck'], axis=1)
)
trials['right_stim_choice_val'] = (
    trials['beta_rl20'] * trials.apply(lambda row: row[f'stim{row["right_stim"]}_value_rl'], axis=1) +
    trials['beta_ck20'] * trials.apply(lambda row: row[f'stim{row["right_stim"]}_value_ck'], axis=1)
)

In [62]:
def choice_prob(right, left):
    exp_right = np.exp(right).values
    exp_left = np.exp(left).values
    return (exp_right, exp_left) / (exp_left + exp_right)


In [66]:
choice_probs = choice_prob(trials['right_stim_choice_val'], trials['left_stim_choice_val'])

In [67]:
trials.loc[:,['choice_prob_right','choice_prob_left']]

,choice_prob_right,choice_prob_left
0,0.786844,0.213156
1,0.982990,0.017010
2,0.982990,0.017010
3,0.011275,0.988725
4,0.887179,0.112821
...,...,...
323,0.036980,0.963020
324,0.125106,0.874894
325,0.268436,0.731564
326,0.511737,0.488263


In [68]:
choice_probs[0]

array([7.86843766e-01, 9.82990298e-01, 5.57207177e-01, 1.12749042e-02,
       8.87178842e-01, 4.42792823e-01, 8.14960931e-01, 9.02573100e-01,
       2.00381529e-01, 8.38557605e-01, 9.28461237e-01, 6.10074440e-01,
       8.35392097e-02, 8.35392097e-02, 1.09509305e-01, 6.85513823e-01,
       2.82157534e-01, 1.33832838e-01, 5.03954830e-01, 8.86211614e-01,
       8.27960798e-02, 6.38514007e-01, 2.21365694e-01, 2.65190012e-01,
       1.78413403e-01, 2.16780706e-01, 9.21289213e-01, 8.41801568e-01,
       6.38420992e-02, 2.12441960e-01, 7.44527143e-02, 7.33729754e-01,
       9.17697464e-01, 9.54783118e-02, 1.14443328e-01, 1.66758271e-01,
       8.64871838e-01, 1.43824434e-01, 1.85892946e-01, 9.33839606e-01,
       1.39538805e-01, 1.03462337e-01, 8.79293617e-01, 8.30063818e-02,
       7.38142203e-02, 1.20706383e-01, 8.70555604e-01, 9.32233110e-02,
       7.01755051e-02, 8.44434591e-02, 9.23003837e-01, 9.33452556e-01,
       1.18755549e-01, 9.06177246e-01, 9.04491103e-01, 8.73361274e-01,
      

In [45]:
trials.loc[trials['action'].isna(),['choice_prob_right','choice_prob_left']]


,choice_prob_right,choice_prob_left
2,0.982990,0.017010
13,0.083539,0.916461
42,0.103462,0.896538
96,0.068374,0.931626
123,0.081089,0.918911
215,0.061524,0.938476


In [81]:
trials.loc[~((trials['choice_prob_right'].diff()==0) == trials['action'].isna())]

,block,left_stim,right_stim,left_value,right_value,shift,action,rt,chosen_stim,reward,...,reward1C,stim_diff_prop_trainingS,stim_diff_prop_training_chosenS,diff_valS,diff_val_chosenS,score_EHI,first_stim_choice_val,second_stim_choice_val,left_stim_choice_val,right_stim_choice_val
96,learning2,8,7,5,4,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10.430590,13.077797,13.077797,10.430590
97,learning2,3,1,2,1,0,1.0,0.409929,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.758341,5.370286,5.370286,2.758341


In [79]:
trials[90:100]

,block,left_stim,right_stim,left_value,right_value,shift,action,rt,chosen_stim,reward,...,reward1C,stim_diff_prop_trainingS,stim_diff_prop_training_chosenS,diff_valS,diff_val_chosenS,score_EHI,first_stim_choice_val,second_stim_choice_val,left_stim_choice_val,right_stim_choice_val
90,learning1,5,3,3,2,0,1.0,0.374013,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,5.371930,7.512096,7.512096,5.371930
91,learning1,6,4,4,3,1,1.0,0.495369,6.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,9.993110,8.004787,9.993110,8.004787
92,learning1,5,7,3,4,0,2.0,0.722157,7.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,10.315017,7.570397,7.570397,10.315017
93,learning1,6,8,4,5,1,2.0,0.562463,8.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,10.048221,12.962588,10.048221,12.962588
94,learning1,7,8,4,5,0,2.0,0.530107,8.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,13.021769,10.373022,10.373022,13.021769
95,learning1,7,5,4,3,1,1.0,0.518548,7.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,10.376952,7.575089,10.376952,7.575089
96,learning2,8,7,5,4,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10.430590,13.077797,13.077797,10.430590
97,learning2,3,1,2,1,0,1.0,0.409929,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.758341,5.370286,5.370286,2.758341
98,learning2,5,7,3,4,1,2.0,0.597076,7.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,7.578372,10.430590,7.578372,10.430590
99,learning2,8,6,5,4,0,1.0,0.564293,8.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,10.050316,13.077797,13.077797,10.050316
